In [1]:
# Importing libraries
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import numpy as np
from scipy.stats import boxcox

In [2]:
# Global settings
global experiment_run
global components
global prefix_f
global d_folder
global load

In [3]:
# Class to load data from experiments
class LoadData:

    __num_rows = 0
    __s_folder = None
    __algo = None

    def __init__(self, num_rows: int, s_folder: str, algo: str):
        self.__num_rows = num_rows
        self.__s_folder = s_folder
        self.__algo = algo

    def load_run_table(self):
        try:
            run_table = pd.read_csv(f'{self.__s_folder}/run_table.csv', nrows=self.__num_rows)
            return run_table
        except:
            return None
        
    def log_transform(self, df, label):
        return df[label].apply(lambda x: np.log(x + 1) if x > 0 else np.nan)

    def boxcox_transform(self, df, label):
        return boxcox(df[label].dropna() + 1)
    
    def filter_df(self, df, label, value):
        return df[df[label] == value]
    
    def group_df_by(self, df, key, outliers):
        grouped_df = df.groupby(key)
        if not outliers:
            return grouped_df
        else:
            cleaned_df = grouped_df.apply(self.remove_outliers, column_name='avg_energy_pct')
            cleaned_df = cleaned_df.rename(columns={key: f'new_{key}'})
            #cleaned_df.reset_index()
            regrouped_df = cleaned_df.groupby(key)
            return regrouped_df
            
    def load_nav2_success(self, component, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        energy_files = {}
        sum_success_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            if row['__done'] == 'TODO':
                continue
            folder_path = f"{self.__s_folder}/{run_id}"
            file_path = os.path.join(folder_path, 'nav2_performance.csv')
            if file_path:
                try:
                    nav2_df = pd.read_csv(file_path)
                    nav2_df['success'] = pd.to_numeric(nav2_df['success'])
                    success = nav2_df['success'].sum()
                    runs_data[run_id] = success
                except Exception as e:
                    print(f"Error processing file for run_id {run_id}: {e}")
                    runs_data[run_id] = 0

        sum_success_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'success'])
        merged_df = df.merge(sum_success_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df
    
    def load_nav2_time(self, component, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        nav_time_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            if row['__done'] == 'TODO':
                continue
            folder_path = f"{self.__s_folder}/{run_id}"
            file_path = os.path.join(folder_path, 'nav2_performance.csv')
            if file_path:
                try:
                    nav2_df = pd.read_csv(file_path)
                    nav2_df['navigation_time'] = pd.to_numeric(nav2_df['navigation_time'], errors='coerce')
                    nav_time = nav2_df['navigation_time'].sum()
                    if nav_time == 0:
                        nav_time = 120
                    runs_data[run_id] = nav_time
                except Exception as e:
                    # print(f"Error processing file for run_id {run_id}: {e}")
                    runs_data[run_id] = 120

        nav_time_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'navigation_time'])
        merged_df = df.merge(nav_time_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df
    
    def load_nav2_path_length(self, component, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        nav_time_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            if row['__done'] == 'TODO':
                continue
            folder_path = f"{self.__s_folder}/{run_id}"
            file_path = os.path.join(folder_path, 'nav2_performance.csv')
            if file_path:
                try:
                    nav2_df = pd.read_csv(file_path)
                    nav2_df['planned_distance_m'] = pd.to_numeric(nav2_df['planned_distance_m'], errors='coerce')
                    nav_time = nav2_df['planned_distance_m'].sum()
                    runs_data[run_id] = nav_time
                except Exception as e:
                    # print(f"Error processing file for run_id {run_id}: {e}")
                    runs_data[run_id] = 0

        nav_time_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'planned_distance_m'])
        merged_df = df.merge(nav_time_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df

    def load_nav2_recoveries(self, component, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        nav_time_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            if row['__done'] == 'TODO':
                continue
            folder_path = f"{self.__s_folder}/{run_id}"
            file_path = os.path.join(folder_path, 'nav2_performance.csv')
            if file_path:
                try:
                    nav2_df = pd.read_csv(file_path)
                    nav2_df['recoveries'] = pd.to_numeric(nav2_df['recoveries'], errors='coerce')
                    nav_time = nav2_df['recoveries'].sum()
                    runs_data[run_id] = nav_time
                except Exception as e:
                    # print(f"Error processing file for run_id {run_id}: {e}")
                    runs_data[run_id] = 0

        nav_time_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'recoveries'])
        merged_df = df.merge(nav_time_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df

    def load_power(self, component, transform: bool, outliers: bool):
        df = self.load_run_table()
        runs_data = {}
        energy_files = {}
        avg_energy_df = {}
        for index, row in df.iterrows():
            run_id = row['__run_id']
            folder_path = f"{self.__s_folder}/{run_id}"
            energy_files = glob.glob(os.path.join(folder_path, f'pj_{component}_server.csv-*.csv'))
            if energy_files:
                try:
                    energy_df = pd.read_csv(energy_files[0])
                    energy_df['CPU Power'] = pd.to_numeric(energy_df['CPU Power'], errors='coerce')
                    if transform:
                        energy_df['CPU Power'], _ = self.boxcox_transform(energy_df, 'CPU Power')
                    filtered_df = energy_df[energy_df['CPU Power'] > 0.001]
                    avg_energy_pct = filtered_df['CPU Power'].mean()
                    
                    runs_data[run_id] = avg_energy_pct
                except Exception as e:
                    print(f"Error processing file for run_id {run_id}: {e}")

        avg_energy_df = pd.DataFrame(list(runs_data.items()), columns=['__run_id', 'avg_energy_pct'])
        merged_df = df.merge(avg_energy_df, on='__run_id')

        clean_df = merged_df.dropna()

        return clean_df

    def remove_outliers(self, group, column_name):
        Q1 = group[column_name].quantile(0.25)
        Q3 = group[column_name].quantile(0.75)
        
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return group[(group[column_name] >= lower_bound) & (group[column_name] <= upper_bound)]

    def load_all_energy():
        pass

    def load_energy_mean():
        pass

    def load_energy_total():
        pass

    def load_all_cpu():
        pass

    def load_all_memory():
        pass

In [5]:
load = None

In [6]:
# Boxplot for Power Consumption
def gen_power_boxplot(component="controller", group_by="configuration", filter=None, transform=False, outliers=False ):
    data = load.load_power(component, transform, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    grouped = data.groupby(group_by)

    print(data.head())

    boxplot_data = []
    labels = []

    for group_name, group_data in grouped:
        boxplot_data.append(group_data["avg_energy_pct"].values)
        labels.append(group_name)

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.boxplot(data=boxplot_data)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.ylabel("Power (W)")
    plt.xlabel(group_by.capitalize())
    if prefix_graph!="default":
        if component=="controller":
            mean_val = 0.0041
        else:
            mean_val = 0.0016
        plt.axhline(y=mean_val, color="red", linestyle="--", label=f"Default Conf. Mean = {mean_val:.4f}")
        plt.legend()
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"boxplot_{prefix_graph}_{component}_{group_by}_power.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [7]:
# Boxplot for Nav2 Navigation Time
def gen_nav_time_boxplot(component="controller", group_by="configuration", filter=None, outliers=False ):
    data = load.load_nav2_time(component, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    grouped = data.groupby(group_by)

    print(data.head())

    boxplot_data = []
    labels = []

    for group_name, group_data in grouped:
        boxplot_data.append(group_data["navigation_time"].values)
        labels.append(group_name)

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.boxplot(data=boxplot_data)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.ylabel("Navigation Time (s)")
    plt.xlabel(group_by.capitalize())
    if prefix_graph!="default":
        if component=="controller":
            mean_val = 32
        else:
            mean_val = 32
        plt.axhline(y=mean_val, color="red", linestyle="--", label=f"Default Conf. Mean = {mean_val}")
        plt.legend()
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"boxplot_{prefix_graph}_{component}_{group_by}_nav_time.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [8]:
# Boxplot for Nav2 Recoveries
def gen_nav_recoveries(component="controller", group_by="configuration", filter=None, outliers=False ):
    data = load.load_nav2_recoveries(component, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    grouped = data.groupby(group_by)

    print(data.head())

    boxplot_data = []
    labels = []

    for group_name, group_data in grouped:
        boxplot_data.append(group_data["recoveries"].values)
        labels.append(group_name)

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.boxplot(data=boxplot_data)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.ylabel("Recoveries")
    plt.xlabel(group_by.capitalize())
    if prefix_graph!="default":
        mean_val = 0
        plt.axhline(y=mean_val, color="red", linestyle="--", label=f"Default Conf. Mean = {mean_val}")
        plt.legend()
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"boxplot_{prefix_graph}_{component}_{group_by}_nav_recoveries.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [9]:
# Boxplot for Nav2 Path Length
def gen_nav_path_boxplot(component="controller", group_by="configuration", filter=None, outliers=False ):
    data = load.load_nav2_path_length(component, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    grouped = data.groupby(group_by)

    print(data.head())

    boxplot_data = []
    labels = []

    for group_name, group_data in grouped:
        boxplot_data.append(group_data["planned_distance_m"].values)
        labels.append(group_name)

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.boxplot(data=boxplot_data)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.ylabel("Path Distance (m)")
    plt.xlabel(group_by.capitalize())
    if prefix_graph!="default":
        mean_val = 6.49
        plt.axhline(y=mean_val, color="red", linestyle="--", label=f"Default Conf. Mean = {mean_val:.2f}")
        plt.legend()
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"boxplot_{prefix_graph}_{component}_{group_by}_nav_path.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [10]:
def gen_nav2_success_percentage_barplot(component="controller", group_by="configuration", filter=None, outliers=False):
    data = load.load_nav2_success(component, outliers)
    
    if filter:
        key, value = filter
        data = load.filter_df(data, key, value)

    # Group by and compute mean (percentage of success == 1)
    grouped = data.groupby(group_by)["success"].mean().reset_index()
    grouped["success_pct"] = grouped["success"] * 100  # convert to percentage

    print(grouped.head())

    # Plot using Seaborn
    plt.figure(figsize=(4, 4))
    sns.barplot(x=group_by, y="success_pct", data=grouped)
    plt.xticks(rotation=90)
    plt.ylabel("Success (%)")
    plt.xlabel(group_by.capitalize())
    plt.ylim(0, 100)  # percentage scale
    if prefix_graph!="default":
        mean_val = 100
        plt.axhline(y=mean_val, color="red", linestyle="--", label=f"Default Conf. Mean = {mean_val}")
        plt.legend()
    plt.tight_layout()

    output_file = os.path.join(d_folder, f"barplot_{prefix_graph}_{component}_{group_by}_success_percentage.pdf")
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")


In [ ]:
# Boxplot Agregate Energy Consumption
def gen_energy_barplot(group_by=None, filter=None, outliers=False, transform=False,
                       outfile="barplot_energy.pdf", load=None):
   
    # Load data
    df_time = load.load_nav2_time(component=None, outliers=outliers)
    df_ctrl = load.load_power(component="controller", transform=transform, outliers=outliers)
    df_plan = load.load_power(component="planner",   transform=transform, outliers=outliers)

    # Optional filter
    if filter:
        key, value = filter
        df_time = df_time[df_time[key] == value]
        df_ctrl = df_ctrl[df_ctrl[key] == value]
        df_plan = df_plan[df_plan[key] == value]

    # Rename for clarity
    df_ctrl = df_ctrl.rename(columns={"avg_energy_pct": "controller_power"})
    df_plan = df_plan.rename(columns={"avg_energy_pct": "planner_power"})

    # Merge by run_id
    df = (df_time[["__run_id", "navigation_time"] + [c for c in df_time.columns if c not in ["__run_id","navigation_time"]]]
            .merge(df_ctrl[["__run_id", "controller_power"]], on="__run_id", how="inner")
            .merge(df_plan[["__run_id", "planner_power"]],   on="__run_id", how="inner"))

    # Compute energy
    df["energy"] = df["navigation_time"] * (df["controller_power"] + df["planner_power"])

    # Aggregate
    if group_by is None:
        summary = df["energy"].mean()
        plt.figure(figsize=(4, 4))
        sns.barplot(x=["Energy"], y=[summary])
        plt.ylabel("Energy (J)")
        plt.tight_layout()
    else:
        if isinstance(group_by, str):
            group_cols = [group_by]
        else:
            group_cols = list(group_by)

        grouped = df.groupby(group_cols, as_index=False)["energy"].mean(numeric_only=True)

        plt.figure(figsize=(4, 4))
        sns.barplot(x=group_cols[0], y="energy", data=grouped)
        plt.ylabel("Energy (J)")
        plt.xlabel(group_cols[0].capitalize())
        plt.xticks(rotation=90)
        if prefix_graph!="default":
            mean_val = 0.17
            if prefix_graph=="large_map":
                mean_val = mean_val * 2
            plt.axhline(y=mean_val, color="red", linestyle="--", label=f"Default Conf. Mean = {mean_val:.2f}")
            plt.legend()
        plt.tight_layout()

    # Save plot
    output_file = os.path.join(d_folder, outfile)
    plt.savefig(output_file, dpi=300)
    plt.close()
    print(f"Saved: {output_file}")

In [12]:
# Paths
experiment_run = ["small_map", "default", "large_map"]
components = ["controller", "planner"]
prefix_f = {"small_map": "data/greenros_reconf_world_small_full_1/",
            "default": "data/greenros_reconf_world_small_fix_config/",
            "large_map": "data/greenros_reconf_world_large/"}
d_folder = "./graphs/"

for prefix_graph in experiment_run:
    prefix = prefix_f[prefix_graph]
    s_folder = prefix_f[prefix_graph]
    std_folder = s_folder
    # Ensure the output folder exists
    os.makedirs(d_folder, exist_ok=True)

    # Load Runtable
    load = LoadData(num_rows=175, s_folder=s_folder, algo="all")
    
    for component in components:
        print(f"Generating boxplot for {component} - {prefix_graph}")
        # Power Boxplot
        gen_power_boxplot(component=component, group_by="configuration", filter=None, transform=False, outliers=True)
        # Nav2 Time Boxplot
        gen_nav_time_boxplot(component=component, group_by="configuration", filter=None, outliers=True)
        # Nav2 Recoveries Boxplot
        gen_nav_recoveries(component=component, group_by="configuration", filter=None, outliers=True)
        # Nav2 Path Length Boxplot
        gen_nav_path_boxplot(component=component, group_by="configuration", filter=None, outliers=True)
        # Nav2 Success Percentage Barplot
        gen_nav2_success_percentage_barplot(component=component, group_by="configuration", filter=None, outliers=True)
        # Energy Consumption Barplot
        gen_energy_barplot(group_by="configuration", filter=None, outliers=True, transform=False,
                       outfile=f"barplot_{prefix_graph}_energy.pdf", load=load)



Generating boxplot for controller - small_map
  __run_id __done  round  configuration  position_goal  number_obstacles  \
0    run_0   DONE      0              0              2                 0   
1    run_1   DONE      0              0              2                 2   
2    run_2   DONE      0              1              2                 0   
3    run_3   DONE      0              1              2                 2   
4    run_4   DONE      0              2              2                 0   

   avg_energy_pct  
0        0.003192  
1        0.003791  
2        0.003337  
3        0.001971  
4        0.002755  
Saved: ./graphs/boxplot_small_map_controller_configuration_power.pdf
  __run_id __done  round  configuration  position_goal  number_obstacles  \
0    run_0   DONE      0              0              2                 0   
1    run_1   DONE      0              0              2                 2   
2    run_2   DONE      0              1              2                 0   
3   